<a href="https://colab.research.google.com/github/joshbarrass/stable-diffusion-colab-launcher/blob/master/Stable_Diffusion_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download the web UI...
%cd /content/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd /content/stable-diffusion-webui
# ...checkout current latest commit at time of writing (2023/02/28) to keep repo stable...
!git checkout 0cc0ee1

!pip install huggingface_hub
from huggingface_hub import hf_hub_url as hf_url

# helper classes for downloads
# download helper classes can implement any __init__ interface to accept
# any necessary args, but must implement a get_url(self) which return a
# download URL. The URL returned by this function is passed to curl to
# download the requested model.
class HFDownload:
  def __init__(self, repo_id, filename, subfolder=None, repo_type=None, revision=None):
    self.repo_id = repo_id
    self.filename = filename
    self.subfolder = subfolder
    self.repo_type = repo_type
    self.revision = revision
  
  def get_url(self):
    return hf_url(repo_id=self.repo_id,
                  filename=self.filename,
                  subfolder=self.subfolder,
                  repo_type=self.repo_type,
                  revision=self.revision)

class CurlDownload:
  def __init__(self, url):
    self.url = url
  
  def get_url(self):
    return self.url

# define model download options
AVAILABLE_MODELS = {
    "AbyssOrangeMix2 SFW": HFDownload("WarriorMama777/OrangeMixs",
                                      "AbyssOrangeMix2_sfw.safetensors",
                                      subfolder="Models/AbyssOrangeMix2"),
    "Ally's Mix": CurlDownload("https://civitai.com/api/download/models/1259?type=Model&format=SafeTensor"),
    "Ally's Mix II: Churned": CurlDownload("https://civitai.com/api/download/models/4274?type=Model&format=SafeTensor"),
    "Ally's Mix III: Revolutions": CurlDownload("https://civitai.com/api/download/models/12763?type=Model&format=SafeTensor"),
    "Cheese Daddy's Landscapes Mix v3.5": CurlDownload("https://civitai.com/api/download/models/7304?type=Model&format=SafeTensor"),
    "Cheese Daddy's Landscapes Mix v2.0 Deliberate": CurlDownload("https://civitai.com/api/download/models/5929?type=Model&format=SafeTensor"),
    "DreamShaper v3.2": HFDownload("Lykon/DreamShaper",
                                   "Dreamshaper_3.32_baked_vae_clip_fix.safetensors"),
    "GTM_UltimateBlend_v3": HFDownload("GalaxyTimeMachine/GTM_UltimateBlend_v3",
                                       "GTM_UltimateBlend_v3.safetensors"),
    "UnstableInkDreamV6": HFDownload("Oosayam/UnstableSamInkDream",
                                     "UnstableinkdreamV6.safetensors"),
    "Waifu Diffusion v1.3": HFDownload("hakurei/waifu-diffusion-v1-3",
                                       "wd-v1-3-float32.ckpt"),
    "YoShin Diffusion": CurlDownload("https://civitai.com/api/download/models/994"),
}

#@markdown # Stable Diffusion Colab Launcher
#@markdown Please read these instructions before running this cell.
#@markdown ## Step 1
#@markdown Ensure the notebook is a GPU notebook. 
#@markdown This should have been selected automatically, but you can check this by going to the top of the page and selecting `Runtime > Change runtime type`. The Hardware Accelerator should be listed as GPU. If it is not, change this now.
#@markdown ## Step 2
#@markdown Select the model you want from the dropdown below. If you're not sure which model to choose, you can search the model name on [civitai.com](https://civitai.com/) to get a better idea of what each model is like and some example prompts.

# TODO: can this be populated automatically?
CHOSEN_MODEL = "Ally's Mix" #@param ["AbyssOrangeMix2 SFW", "Ally's Mix", "Ally's Mix II: Churned", "Ally's Mix III: Revolutions", "Cheese Daddy's Landscapes Mix v2.0 Deliberate", "Cheese Daddy's Landscapes Mix v3.5", "DreamShaper v3.2", "GTM_UltimateBlend_v3", "UnstableInkDreamV6", "Waifu Diffusion v1.3", "YoShin Diffusion"]

#@markdown ## Step 3
#@markdown Run this cell! Either press CTRL+Enter or SHIFT+Enter, or click the Play symbol (⏵) to the left of this cell.
#@markdown 
#@markdown This cell will produce a lot of output in the console, and it may take a few minutes for the model to be downloaded and the UI to be set up. Please be patient!
#@markdown 
#@markdown When the model is ready, a gradio link will appear in the console. Click this link and enjoy!
#@markdown 
#@markdown ## Choosing a Different Model
#@markdown If you want to download and run a different model, simply press the Stop symbol (⏹) to the left of this cell and go through these steps again, starting from Step 2. When you have downloaded multiple models in this way, there will be a dropdown menu in the top left corner of the gradio interface where you can switch between the models you have downloaded. Check here to make sure you're using the model you want, and you can switch between these models at will without stopping the Colab notebook.
#@markdown 
#@markdown ## Finishing Up
#@markdown When you are done using the model, make sure to delete your session, as this will help avoid computing resources going to waste.
#@markdown 
#@markdown Click the ▾ in the top right and select `Disconnect and delete runtime`. You're then free to close the tab :)

# download the model
import os, sys
DOWNLOAD_URL = AVAILABLE_MODELS[CHOSEN_MODEL].get_url()
%cd models/Stable-diffusion
!wget -nc --content-disposition "$DOWNLOAD_URL" && touch /tmp/downloaded_model
if not os.path.exists("/tmp/downloaded_model"):
  print("[\x1b[31mERROR\x1b[0m] Failed to download the model. Run this cell again to try again.\n        If the problem persists, please try a different model, as the model\n        may no longer be available, and please file an issue on GitHub.")
  sys.exit(1)
os.remove("/tmp/downloaded_model")

# start web ui
# to get xformers on colab (for extra speed), must request it with --xformers
%cd /content/stable-diffusion-webui
!python launch.py --share --xformers